In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timezone
import plotly.express as px
import plotly.graph_objects as go
from plotly.io import to_html

import requests
import json
from tqdm import tqdm

In [15]:
agg_df = pd.read_csv('../hdb_carpark_info/agg_df.csv')

In [18]:
def get_plot(agg_df, cp, selected_datetime):
    dow = selected_datetime.weekday()
#     dow = selected_datetime.dayofweek
    h = selected_datetime.hour
    m = selected_datetime.minute
    motd = h*60 + m

    cp_msk = agg_df['carpark_number'] == cp
    dow_msp = agg_df['dayOfWeek'] == dow

    temp = agg_df[cp_msk & dow_msp]

    selected_idx = np.where(temp['minsOfTheDay']==motd)[0][0]

    if selected_idx<10:
        left_idx = 0
        right_idx = 21

        left_highlight_idx = 0
        right_highlight_idx = selected_idx+5

    elif selected_idx>=temp.shape[0]-10:
        left_idx = temp.shape[0]-21
        right_idx = temp.shape[0]

        left_highlight_idx = selected_idx-5
        right_highlight_idx = temp.shape[0]

    else:
        left_idx = selected_idx-10
        right_idx = selected_idx+11

        left_highlight_idx = selected_idx-5
        right_highlight_idx = selected_idx+5    

    X = temp.iloc[left_idx:right_idx]['minsOfTheDay']
    Y = temp.iloc[left_idx:right_idx]['lots_available']


    left_timeDelta = pd.timedelta_range(start='5 min', periods=selected_idx-left_idx, freq='5min')[::-1]
    right_timeDelta =pd.timedelta_range(start='0 day', periods=right_idx-selected_idx, freq='5min')
    X = [selected_datetime-td for td in left_timeDelta] + [selected_datetime+td for td in right_timeDelta]
#     print('X: ',X)
#     print('left_timeDelta: ', left_timeDelta)
#     print('right_timeDelta: ', right_timeDelta)
    left_timeDelta = pd.timedelta_range(start='5 min', periods=selected_idx-left_highlight_idx, freq='5min')[::-1]
    right_timeDelta =pd.timedelta_range(start='0 day', periods=right_highlight_idx-selected_idx, freq='5min')
    X_highlight = [selected_datetime-td for td in left_timeDelta] + [selected_datetime+td for td in right_timeDelta]
    
#     X_highlight = temp.iloc[left_highlight_idx:right_highlight_idx]['minsOfTheDay']
    max_availability = Y.max()


    
    fig = go.Figure(go.Scatter(x=X_highlight, y=[max_availability+5 for _ in range(len(X_highlight))], 
                               fill="tozeroy", marker = dict(size=1,
                                  line=dict(width=1,
                                            color='DarkSlateGrey')),
                              name='Relevant Time Window'),
                   layout=go.Layout(
        title=go.layout.Title(text="Forecast availability on Date: {}".format(selected_datetime))
    ))
    
#     print('len(X): ', len(X))
    fig.add_trace(go.Scatter(x=X, y=Y,
                        mode='lines',
                        name='Available Lots'))
    
    
    fig.show()
    out_html = to_html(fig, include_plotlyjs=False, full_html=False)
    return out_html

In [19]:
selected_datetime = pd.datetime.now()


<ipython-input-19-9045a05ab31c>:1: FutureWarning:

The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.



In [20]:
selected_datetime.dt.round('1s')

AttributeError: 'datetime.datetime' object has no attribute 'dt'

In [23]:
selected_datetime = pd.to_datetime('2021-01-30 13:50')
cp = 'YHS'
get_plot(agg_df, cp, selected_datetime)

'<div>                            <div id="01d6d2f1-cb48-4fda-b5fe-badc1e509781" class="plotly-graph-div" style="height:100%; width:100%;"></div>            <script type="text/javascript">                                    window.PLOTLYENV=window.PLOTLYENV || {};                                    if (document.getElementById("01d6d2f1-cb48-4fda-b5fe-badc1e509781")) {                    Plotly.newPlot(                        "01d6d2f1-cb48-4fda-b5fe-badc1e509781",                        [{"fill": "tozeroy", "marker": {"line": {"color": "DarkSlateGrey", "width": 1}, "size": 1}, "name": "Relevant Time Window", "type": "scatter", "x": ["2021-01-30T13:25:00", "2021-01-30T13:30:00", "2021-01-30T13:35:00", "2021-01-30T13:40:00", "2021-01-30T13:45:00", "2021-01-30T13:50:00", "2021-01-30T13:55:00", "2021-01-30T14:00:00", "2021-01-30T14:05:00", "2021-01-30T14:10:00"], "y": [25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0, 25.0]}, {"mode": "lines", "name": "Available Lots", "type": "scatter